# QUBO Formulation of Travelling Salesman Problem

## QUBO formula

$$F(X_1, X_2, ... , X_N) =\sum_{i=1}^N c_i X_i +\sum_{i=1}^N \sum_{j=1}^i Q_{ij} \times X_i \times X_j$$
with $$X_i \in \{0,1\}$$ and $$c_i, Q_{ij} \in R$$

**QUBO minimisation:**
- $N$ is the number of decision variables (problem size)
- $X_i$ are the decision variables
- $c_i, Q_{ij}$ are given parameters identifying the QUBO instance (coefficients of linear and quadratic terms respectively)
- $F$ is the objective function to minimise

**QUBO example:**

$$F(X_1, X_2, X_3, X_4) = 5 X_1 -3 X_2 -8 X_3 -6 x_4 + 4 X_1X_2 + 8 X_1 X_3 - 2 X_2 X_3 + 10 X_3 X_4$$

## QUBO formulation steps

**INPUT: generic constrained combinatorial optimisation problem**

1. Non-binary optimisation problem: binary encoding of decision variables
2. Constrained optimisation problem: use penalty functions to embed constraints in the objective function
3. Non-polynomial objective function: transform the objective function to a polynomial
4. Non-quadratic objective function: reduce the degree of the polynomial from higher order to quadratic

**OUTPUT: QUBO formulation of input problem**





## Travelling Salesman Problem

### Problem Statement

The travelling salesman problem asks the following question: "Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city and returns to the origin city?" 


### Direct High-Level Formulation 

Solutions are tours represented as permuations whose tour lenght is to minimise:

minimize f(x) = $(\sum_{i=1}^{n-1} d_{x_i,x_{i+1}}) + d_{x_n,x_1}$ (tour lenght)

subject to (x is a permutation):

$x \in [1, n]^n$ (x is an integer vector of size n with entries between 1 and n) 

$\forall i, j \in [1, n]$ with $i < j: x_i \neq x_j$ (all entries of x are unique)

### TSP working example: High-Level Formulation

In [1]:
import random
from pprint import pprint
import numpy as np
from numpy import array, dot
from numpy.linalg import inv
from itertools import combinations, product, permutations
from numpy import argmax

<img src="https://i2.wp.com/www.techiedelight.com/wp-content/uploads/1.png?zoom=2.5&resize=270%2C282&ssl=1" width="200">

In [2]:
### Problem Instance (see above)

n = 4 # number of cities

dist = [[0, 10, 50, 45], [10, 0, 25, 25], [50, 25, 0, 40], [45, 25, 40, 0]]

### Minimum: (A,B,C,D), tour len: 120


In [3]:
### High-level Formulation

# tour len
def objective_function(x): return sum(dist[x[i]][x[(i+1) % n]] for i in range(n))  # to minimise

# domain: x is a permutation of 1 ... n


In [4]:
### Random solution
    
x = np.random.permutation([0, 1, 2, 3])

#x = [0, 1, 2, 3] # optimum

x, objective_function(x) 


(array([0, 1, 3, 2]), 125)

In [5]:
### Find Optimum by Enumeration

min([ (x, objective_function(x)) for x in permutations(range(n), n) ], key=lambda y: y[1])

((0, 1, 2, 3), 120)

This is a correct global minimum (A-B-C-D)

### 1. Binary Representation (Dantzig-Fulkerson-Johnson formulation)

We use a graph to represent directed tours. Label the cities with the numbers $1$ to $n$ (nodes) and define arcs as follows:

$ x_{ij} = \begin{cases} 1 & \text{the path goes from city } i \text{ to city } j \\ 0 & \text{otherwise} \end{cases} $

Take $c_{ij}$ to be the distance from city $i$ to city $j$ (weights on arcs). Then TSP can be written as the following binary linear programming problem:

$\begin{align}
\min &\sum_{i=1}^n \sum_{j\ne i,j=1}^nc_{ij}x_{ij}\colon &&  \\
     & \sum_{i=1,i\ne j}^n x_{ij} = 1 && j=1, \ldots, n; \\
     & \sum_{j=1,j\ne i}^n x_{ij} = 1 && i=1, \ldots, n; \\
     & \sum_{i \in Q}{\sum_{j \in Q}{x_{ij}}} \leq |Q| - 1 && \forall Q \subseteq \{2, \ldots, n\} \\
\end{align}$

The objective function (to minimise) is the sum of the length of all the arcs in the tour.
The first group of constraints means enter every city $j$ exactly once. The second group of constraints means exit every city $i$ exactly once. The last group of constraints ensures that there are no sub-tours among the non-starting vertices, 
so the solution returned is a single tour and not the union of smaller tours.

### TSP working example: Binary Representation (Dantzig-Fulkerson-Johnson formulation)

This is the instatiation of the DFJ formulation for the example problem instance introduced earlier.

$n = 4$

$\begin{align}
\min & 10 x_{01} + 50 x_{02} + 45 x_{03} + 10 x_{10} + 25 x_{12} + 25 x_{13} + 50 x_{20} + 25 x_{21} + 40 x_{23} + 45 x_{30} + 25 x_{31} + 40 x_{32} \colon && \\
& \\
& x_{10} + x_{20} + x_{30} = 1  \\
& x_{01} + x_{21} + x_{31} = 1  \\
& x_{02} + x_{12} + x_{32} = 1  \\
& x_{03} + x_{13} + x_{23} = 1  \\
& \\
& x_{01} + x_{02} + x_{03} = 1  \\
& x_{10} + x_{12} + x_{13} = 1  \\
& x_{20} + x_{21} + x_{23} = 1  \\
& x_{30} + x_{31} + x_{32} = 1  \\
& \\
& x_{11} \leq 0 \\ %{1}
& x_{22} \leq 0 \\ %{2}
& x_{33} \leq 0 \\ %{3}
& x_{11} + x_{12} + x_{21} + x_{22} \leq 1 \\ %{1,2}
& x_{11} + x_{13} + x_{31} + x_{33} \leq 1 \\ %{1,3}
& x_{22} + x_{23} + x_{32} + x_{33} \leq 1 \\ %{2,3}
& x_{11} + x_{12} + x_{13} + x_{21} + x_{22} + x_{23} + x_{31} + x_{32} + x_{33} \leq 2 \\ %{1,2,3}
\end{align}$

 

### 2. Removing Constraints (Penalty Method)

To transform a constrained optimisation problem into an unconstrained one, we need to embed the constraints in the objective function. This can be done by first wriring the constraints as equality constraints, and then using a penaltie to embed these in the objective function such that the global minimum of the resulting uncontrained problem corresponds to the global minimum of the original constrained problem. This can always be do by choosing suitable large penalty weights for the constraints.

**Non-Negatve Equality Constraints** 

$\begin{align}
\min &\sum_{i=1}^n \sum_{j\ne i,j=1}^nc_{ij}x_{ij}\colon &&  \\
     & \sum_{i=1,i\ne j}^n x_{ij} = 1 && \rightarrow (\sum_{i=1,i\ne j}^n x_{ij} - 1)^2=0 \\
     & \sum_{j=1,j\ne i}^n x_{ij} = 1 && \rightarrow (\sum_{j=1,j\ne i}^n x_{ij} - 1)^2=0 \\
     & \sum_{i \in Q}{\sum_{j \in Q}{x_{ij}}} \leq |Q| - 1 &&  \rightarrow (\sum_{i \in Q}{\sum_{j \in Q}{x_{ij}}} - |Q| + 1 + Y)^2=0 \\
\end{align}$

Added slack variable $Y$ to convert inequality to equality.

**Unconstraned problem**

$\begin{align}
\min &\sum_{i=1}^n \sum_{j\ne i,j=1}^nc_{ij}x_{ij} +  \\
     & A (\sum_{i=1,i\ne j}^n x_{ij} - 1)^2 + \\
     & B (\sum_{j=1,j\ne i}^n x_{ij} - 1)^2 + \\
     & C (\sum_{i \in Q}{\sum_{j \in Q}{x_{ij}}} - |Q| + 1 + Y)^2 \\
\end{align}$

For suitably large positive penalty weights $A$, $B$ and $C$.

In some cases, penalty factors can be chosen analytically using mathematical properties of the problem at hand. In other cases, they need to be determined experimentally (by sequential trial and error). The aim is to find the smallest factors that do not lead to infeasible optima or near-optimal solutions.

### TSP working example: Removing Constraints (Penalty Method)

**Non-Negatve Equality Constraints**

$\begin{align}
\min & 10 x_{01} + 50 x_{02} + 45 x_{03} + 10 x_{10} + 25 x_{12} + 25 x_{13} + 50 x_{20} + 25 x_{21} + 40 x_{23} + 45 x_{30} + 25 x_{31} + 40 x_{32} \colon && \\
& \\
& (x_{10} + x_{20} + x_{30} - 1)^2 = 0 \\
& (x_{01} + x_{21} + x_{31} - 1)^2 = 0 \\
& (x_{02} + x_{12} + x_{32} - 1)^2 = 0 \\
& (x_{03} + x_{13} + x_{23} - 1)^2 = 0 \\
& \\
& (x_{01} + x_{02} + x_{03} - 1)^2 = 0 \\
& (x_{10} + x_{12} + x_{13} - 1)^2 = 0 \\
& (x_{20} + x_{21} + x_{23} - 1)^2 = 0 \\
& (x_{30} + x_{31} + x_{32} - 1)^2 = 0 \\
& \\
& x_{11}^2 = 0 \\
& x_{22}^2 = 0 \\
& x_{33}^2 = 0 \\
& (x_{11} + x_{12} + x_{21} + x_{22} + y_0 - 1)^2 = 0 \\ 
& (x_{11} + x_{13} + x_{31} + x_{33} + y_1 - 1)^2 = 0 \\ 
& (x_{22} + x_{23} + x_{32} + x_{33} + y_2 - 1)^2 = 0 \\ 
& (x_{11} + x_{12} + x_{13} + x_{21} + x_{22} + x_{23} + x_{31} + x_{32} + x_{33} + y_3 + y_4 - 2)^2 = 0 \\
\end{align}$

where $y_0 \cdot y_4$ are slack variables (additional binary decision variables that allow to convert inequality constraints into equality constraints).

**Unconstraned problem**

$\begin{align}
\min & 10 x_{01} + 50 x_{02} + 45 x_{03} + 10 x_{10} + 25 x_{12} + 25 x_{13} + 50 x_{20} + 25 x_{21} + 40 x_{23} + 45 x_{30} + 25 x_{31} + 40 x_{32} + \\
& \\
& A \cdot (x_{10} + x_{20} + x_{30} - 1)^2 + \\
& A \cdot (x_{01} + x_{21} + x_{31} - 1)^2 + \\
& A \cdot (x_{02} + x_{12} + x_{32} - 1)^2 + \\
& A \cdot (x_{03} + x_{13} + x_{23} - 1)^2 + \\
& \\
& B \cdot (x_{01} + x_{02} + x_{03} - 1)^2 + \\
& B \cdot (x_{10} + x_{12} + x_{13} - 1)^2 + \\
& B \cdot (x_{20} + x_{21} + x_{23} - 1)^2 + \\
& B \cdot (x_{30} + x_{31} + x_{32} - 1)^2 + \\
& \\
& C \cdot x_{11}^2 + \\
& C \cdot x_{22}^2 + \\
& C \cdot x_{33}^2 + \\
& C \cdot (x_{11} + x_{12} + x_{21} + x_{22} + y_0 - 1)^2 + \\
& C \cdot (x_{11} + x_{13} + x_{31} + x_{33} + y_1 - 1)^2 + \\ 
& C \cdot (x_{22} + x_{23} + x_{32} + x_{33} + y_2 - 1)^2 + \\
& C \cdot (x_{11} + x_{12} + x_{13} + x_{21} + x_{22} + x_{23} + x_{31} + x_{32} + x_{33} + y_3 + y_4 - 2)^2 \\
\end{align}$

Choice of $A$, $B$ and $C$: since the contraints are integer-valued, we can choose $A=B=C=UB(TourLen)-LB(TourLen)+1$ to guarantee that *any* feasible solution has lower objective function than *any* infeasible solution (this in turn guaratees that the global minimum of the original constrained problem is the same as the global minima of the new unconstrained problem). Simple LB and UB for the TSP are obtained by using the sum of the legths of the $n$ shortest edges and $n$ largest edges, respectively. So, for this specific problem insance we have $LB=10+25+25+40=100$, $UB=50+45+40+35=170$ and $A=B=C=170-100+1=71$. 

In [44]:
### Unconstrained Binary Formulation

# penalty weights
A = B = C = 71

# tour len to minimise
def objective_function(x, y):
    
    return \
    \
    10*x[0,1] + 50*x[0,2] + 45*x[0,3] + 10*x[1,0] + 25*x[1,2] + 25*x[1,3] + 50*x[2,0] + 25*x[2,1] + 40*x[2,3] + 45*x[3,0] \
    + 25*x[3,1] + 40*x[3,2] + \
    \
    A * (x[1,0] + x[2,0] + x[3,0] - 1)**2 + \
    A * (x[0,1] + x[2,1] + x[3,1] - 1)**2 + \
    A * (x[0,2] + x[1,2] + x[3,2] - 1)**2 + \
    A * (x[0,3] + x[1,3] + x[2,3] - 1)**2 + \
    \
    B * (x[0,1] + x[0,2] + x[0,3] - 1)**2 + \
    B * (x[1,0] + x[1,2] + x[1,3] - 1)**2 + \
    B * (x[2,0] + x[2,1] + x[2,3] - 1)**2 + \
    B * (x[3,0] + x[3,1] + x[3,2] - 1)**2 + \
    \
    C * x[1,1]**2 + \
    C * x[2,2]**2 + \
    C * x[3,3]**2 + \
    C * (x[1,1] + x[1,2] + x[2,1] + x[2,2] + y[0] - 1)**2 + \
    C * (x[1,1] + x[1,3] + x[3,1] + x[3,3] + y[1] - 1)**2 + \
    C * (x[2,2] + x[2,3] + x[3,2] + x[3,3] + y[2] - 1)**2 + \
    C * (x[1,1] + x[1,2] + x[1,3] + x[2,1] + x[2,2] + x[2,3] + x[3,1] + x[3,2] + x[3,3] + y[3] + y[4] - 2)**2 
    

# domain: decision variables x is a binary matrix nxn, slack variables y is a binary vector of size 5


In [45]:
### Random solution
    
x = np.random.choice([0, 1], size=(n, n))

y = np.random.choice([0, 1], size=(5))

x, y, objective_function(x, y) 


(array([[1, 1, 0, 0],
        [1, 1, 0, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0]]),
 array([1, 0, 0, 0, 1]),
 1870)

In [46]:
### Find Optimum by Enumeration

min([ (np.array(x).reshape((n,n)), objective_function(np.array(x).reshape((n,n)),y)) for x in product([0,1], repeat=n**2) for y in product([0,1], repeat=5)], key=lambda k: k[1])

(array([[0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0]]),
 120)

This is a correct global minimum (A-D-C-B)

### 3. Transform the objective function to a polynomial

The unconstrained problem is not expressed as a polynomial in $x_{ij}$. To do so, the squares need expansion and then the resulting expression collected so that it is expressed as monomials involving $x_{ij}$. This can be done using computer algebra libraries e.g., Sympy in Python. To get a numeric expression for the coefficients, we need to substitute numerical values for $n$, $c_{ij}$ and $|Q|$. 

### TSP working example: Transform the objective function to a polynomial

The code below get a sybolic expression of the unconstrained problem for the working example, and put it in a polynomial form. 

The obtained polynomial is a generic symbolic polynomial, with the further assumption that the decision variables are binary (i.e., psuedo-Boolean polynomial), the squared variables should be reduced to linear variables. 

In [48]:
from sympy import MatrixSymbol, Matrix, poly
X = MatrixSymbol('X', n, n)
Y = MatrixSymbol('Y', 1, 5)
sym_objective_function = poly(objective_function(X, Y))
sym_objective_function

Poly(142*X[0, 1]**2 + 142*X[0, 1]*X[0, 2] + 142*X[0, 1]*X[0, 3] + 142*X[0, 1]*X[2, 1] + 142*X[0, 1]*X[3, 1] - 274*X[0, 1] + 142*X[0, 2]**2 + 142*X[0, 2]*X[0, 3] + 142*X[0, 2]*X[1, 2] + 142*X[0, 2]*X[3, 2] - 234*X[0, 2] + 142*X[0, 3]**2 + 142*X[0, 3]*X[1, 3] + 142*X[0, 3]*X[2, 3] - 239*X[0, 3] + 142*X[1, 0]**2 + 142*X[1, 0]*X[1, 2] + 142*X[1, 0]*X[1, 3] + 142*X[1, 0]*X[2, 0] + 142*X[1, 0]*X[3, 0] - 274*X[1, 0] + 284*X[1, 1]**2 + 284*X[1, 1]*X[1, 2] + 284*X[1, 1]*X[1, 3] + 284*X[1, 1]*X[2, 1] + 284*X[1, 1]*X[2, 2] + 142*X[1, 1]*X[2, 3] + 284*X[1, 1]*X[3, 1] + 142*X[1, 1]*X[3, 2] + 284*X[1, 1]*X[3, 3] + 142*X[1, 1]*Y[0, 0] + 142*X[1, 1]*Y[0, 1] + 142*X[1, 1]*Y[0, 3] + 142*X[1, 1]*Y[0, 4] - 568*X[1, 1] + 284*X[1, 2]**2 + 284*X[1, 2]*X[1, 3] + 284*X[1, 2]*X[2, 1] + 284*X[1, 2]*X[2, 2] + 142*X[1, 2]*X[2, 3] + 142*X[1, 2]*X[3, 1] + 284*X[1, 2]*X[3, 2] + 142*X[1, 2]*X[3, 3] + 142*X[1, 2]*Y[0, 0] + 142*X[1, 2]*Y[0, 3] + 142*X[1, 2]*Y[0, 4] - 685*X[1, 2] + 284*X[1, 3]**2 + 142*X[1, 3]*X[2, 1] + 

### 4. Reduce the degree of the polynomial if not quadratic

Any pseudo-boolean function can be expressed as a polynomial. However the resulting polynomial may be of degree larger than two i.e., not a QUBO. There are standard mathematical reduction methods to transform higher order polynomials to quadratic polynomials, trading lower order for additional variables. Many optimisation problems do not need this step, i.e., are naturally quadratic (being a little careful with the formulation). But some problems are naturally higher order, e.g., cubic. TSP as formulated above is naturally quadratic, and it does not need further reduction.  

### TSP working example: Reduce the degree of the polynomial if not quadratic

The obtained pseudo-Boolean polynmial is quadratic so it is a QUBO and no degree reduction is required.

### Solving a QUBO and Solution Validation

Once we have a QUBO for a given problem, it can be solved using a classical computer using e.g., stochastic search methods as Simulated Annealing, Tabu Search, and Genetic Algorithms. Or it can be solved on Quantum Computers both Annealers and Gate-Based Architectures (via QAOA). Both classic and quantum solvers have parameters that need tuning. When solving a QUBO on a quantum computer, one may need extra steps e.g., like embedding the QUBO to the physical topology of the machine (Chimera topology for D-wave). Also, quantum computers have a limited number of qbits, and each qbit corresponds to a binary decision variable. So, problem of size larger than the quantum machine needs to be split in separate smaller problems of suitable size. Solved sepratately and then the partial solutions stiched together. Luckly, new D-wave software (hybrid solver) takes care automatically of embedding and problem decomposition. Once a solution is reurned, it needs to be checked and make sure that (i) is feasible (ii) makes sense for the original problem, as a QUBO is only a surrogate of the original problem (guaranteed only to have the same global optimum if the modelling is done without errors), which in practice may have important differences from the original problem.        

### TSP working example: Solving a QUBO and Solution Validation

The obtained QUBO in symbolic form is wrapped into a callable objective functon. The global optimum computed using the QUBO is the same as the original TSP problem. This validates that the obtained QUBO formulaton is correct.


In [49]:
### Wrap symbolic objective function into a function -- very inefficient evaluation!

### TODO: For efficiency: much better extracting all coefficients of the polynomial, organise them in a matrix,
### and returns a new function that computes the objective function by numerical matrix multiplication

def objective_function(x, y):
    
    return sym_objective_function.subs(X, Matrix(x)).subs(Y, Matrix([y])) 


In [50]:
### Random solution
    
x = np.random.choice([0, 1], size=(n, n))

y = np.random.choice([0, 1], size=(5))

x, y, objective_function(x, y) 

#x, y


(array([[0, 1, 1, 1],
        [1, 1, 0, 1],
        [1, 1, 0, 0],
        [0, 1, 1, 0]]),
 array([1, 0, 0, 0, 0]),
 2552)

In [51]:
### Find Optimum by Enumeration -- takes long time

min([ (np.array(x).reshape((n,n)), objective_function(np.array(x).reshape((n,n)),y)) for x in product([0,1], repeat=n**2) for y in product([0,1], repeat=5)], key=lambda k: k[1])

(array([[0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0]]),
 120)

This is a correct global minimum (A-D-C-B)